In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import *
from nltk import word_tokenize
import itertools

## Выгрузка данных из датасета

In [52]:
categories = ['comp.graphics', 'comp.os.ms-windows.misc', 'rec.autos']
remove = ['headers', 'footers', 'quotes']
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories=categories, remove=remove)

In [53]:
twenty_train = pd.DataFrame(twenty_train, columns=['data', 'target']).replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
twenty_test = pd.DataFrame(twenty_test, columns=['data', 'target']).replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)

In [54]:
def stemming(data):
    porter_stemmer = PorterStemmer()
    nltk_tokens = word_tokenize(data)
    line = ''
    for word in nltk_tokens:
        line += ' ' + porter_stemmer.stem(word)
    return line

twenty_train.insert(loc=1, column='data_stemmed', value=twenty_train['data'].apply(lambda text: stemming(text)))
twenty_test.insert(loc=1, column='data_stemmed', value=twenty_test['data'].apply(lambda text: stemming(text)))

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.utils._testing import ignore_warnings 
from sklearn.exceptions import FitFailedWarning, ConvergenceWarning 

In [56]:
%%time
parameters = {
    'RandomForestClassifier': {
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__criterion': ['gini','entropy','log_loss'],
        'clf__max_depth': [3,5,10,None]
    },
    'LogisticRegression': {
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__penalty': ['l1','l2'], 
        'clf__C': [0.001,0.01,0.1,1,10,100,1000]
    },
    'KNeighborsClassifier': {
        'vect__max_features': (1000,5000,10000),
        'vect__stop_words': ('english', None),
        'tfidf__use_idf': (True, False),
        'clf__n_neighbors': (1, 3, 5, 10),
        'clf__p': (1, 2)
    }
}

gs = {}
for clf, param in parameters.items():
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', eval(clf)())
    ])
    gs[clf] = GridSearchCV(text_clf, param, n_jobs=-1, error_score=0.0)
    gs[clf].fit(X = twenty_train['data'], y = twenty_train['target'])

C:\Users\nuvik\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
420 fits failed out of a total of 840.
The score on these train-test partitions for these parameters will be set to 0.0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
420 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nuvik\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nuvik\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py", line 406, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\nuvik\AppData\Local\Programs\Python\Python311\Lib

CPU times: total: 17.2 s
Wall time: 2min 52s


In [57]:
for clf, param in parameters.items():
    predicted = gs[clf].predict(twenty_test['data'])
    print(metrics.classification_report(twenty_test.target, predicted, target_names=categories))

                         precision    recall  f1-score   support

          comp.graphics       0.83      0.74      0.78       389
comp.os.ms-windows.misc       0.83      0.76      0.79       394
              rec.autos       0.79      0.94      0.86       396

               accuracy                           0.81      1179
              macro avg       0.82      0.81      0.81      1179
           weighted avg       0.82      0.81      0.81      1179

                         precision    recall  f1-score   support

          comp.graphics       0.84      0.85      0.84       389
comp.os.ms-windows.misc       0.90      0.74      0.81       394
              rec.autos       0.84      0.97      0.90       396

               accuracy                           0.85      1179
              macro avg       0.86      0.85      0.85      1179
           weighted avg       0.86      0.85      0.85      1179

                         precision    recall  f1-score   support

          comp.gra

In [58]:
r = {}
def highlight_max(x, color):

    return np.where(x == np.nanmax(x.to_numpy()), f"color: {color};", None)

total_style = pd.Series("font-weight: bold;", index=[1])

for clf, param in parameters.items():
    predicted = gs[clf].predict(twenty_test['data'])
    
    pd.DataFrame(gs[clf].cv_results_).to_excel('all' + clf + '.xlsx')
    pd.DataFrame(classification_report(predicted, twenty_test.target, output_dict=True)).to_excel('best' + clf + '.xlsx')
    